# <span style="font-width:bold; font-size: 3rem; color:#1EB182;"><img src="../../images/icon102.png" width="38px"></img> **Hopsworks Feature Store** </span><span style="font-width:bold; font-size: 3rem; color:#333;">- Part 01: Feature Backfill</span>

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/logicalclocks/hopsworks-tutorials/blob/master/advanced_tutorials/bitcoin/1_bitcoin_feature_backfill.ipynb)

## 🗒️ This notebook is divided into the following sections:
1. Fetch historical data.
2. Connect to the Hopsworks feature store.
3. Create feature groups and insert them to the feature store.

![tutorial-flow](../../images/01_featuregroups.png)

---

### <span style="color:#ff5f27;"> 📝 Imports</span>

In [ ]:
!pip install -U hopsworks --quiet

!pip install -U unicorn-binance-rest-api --quiet
!pip install -U python-dotenv --quiet
!pip install -U textblob --quiet
!pip install -U vaderSentiment --quiet
!pip install -U tweepy --quiet

# Hosted notebook environments may not have the local features package
import os

def need_download_modules():
    if 'google.colab' in str(get_ipython()):
        return True
    if 'HOPSWORKS_PROJECT_ID' in os.environ:
        return True
    return False

if need_download_modules():
    print("Downloading modules")
    os.system('mkdir -p features')
    os.system('cd features && wget https://raw.githubusercontent.com/logicalclocks/hopsworks-tutorials/master/advanced_tutorials/bitcoin/features/bitcoin_price.py')
    os.system('cd features && wget https://raw.githubusercontent.com/logicalclocks/hopsworks-tutorials/master/advanced_tutorials/bitcoin/features/tweets.py')
else:
    print("Local environment")

In [ ]:
# Uncomment and fill in if you are running on Colab
# os.environ['TWITTER_API_KEY'] = '{YOUR_API_KEY}'
# os.environ['TWITTER_API_SECRET'] = '{YOUR_API_KEY}'
# os.environ['TWITTER_ACCESS_TOKEN'] = '{YOUR_API_KEY}'
# os.environ['TWITTER_ACCESS_TOKEN_SECRET'] = '{YOUR_API_KEY}'

# os.environ['BINANCE_API_KEY'] = '{YOUR_API_KEY}'
# os.environ['BINANCE_API_SECRET'] = '{YOUR_API_KEY}'

In [ ]:
import pandas as pd

from features import bitcoin_price, tweets

---
## <span style="color:#ff5f27;"> 🧙🏼‍♂️ Parsing Data</span>

You will parse timeseries Bitcoin data from Binance using your own credentials, so you have to get a free Binance account and [create API-keys](https://www.binance.com/en/support/faq/360002502072).

Also, you should [contact Twitter](https://developer.twitter.com/en/docs/twitter-api/getting-started/getting-access-to-the-twitter-api) for their API-keys.


#### Don't forget to create an `.env` configuration file inside this directory where all the necessary environment variables will be stored:

`TWITTER_API_KEY = "YOUR_API_KEY"`

`TWITTER_API_SECRET = "YOUR_API_KEY"`

`TWITTER_ACCESS_TOKEN = "YOUR_API_KEY"`

`TWITTER_ACCESS_TOKEN_SECRET = "YOUR_API_KEY"`


`BINANCE_API_KEY = "YOUR_API_KEY"`

`BINANCE_API_SECRET = "YOUR_API_KEY"`

> If you done it after you run this notebook, restart the Python Kernel (because `functions.py` does not have these variables in his namespace).

![](images/api_keys_env_file.png)

### <span style='color:#ff5f27'> 📈 Bitcoin Data parsing

In [ ]:
# we work with tweets newer then '2021-02-05'
df_bitcoin = bitcoin_price.parse_btc_data(
    start_date="2021-02-05", 
    end_date="today",
)

df_bitcoin.reset_index(drop=True,inplace=True)

print()
print(f"Parsed {df_bitcoin.shape[0]} rows.")
print()

df_bitcoin.head(3)

In [ ]:
df_bitcoin_processed = bitcoin_price.process_btc_data(df_bitcoin)
df_bitcoin_processed.tail(3)

> Older records may come with time=11pm or time=9pm, but new ones have time=10pm. Thats because of timezones and daylight saving time. Lets apply this function to make unix column usable.

In [ ]:
df_bitcoin_processed.unix = df_bitcoin_processed.unix.apply(bitcoin_price.fix_unix)

In [ ]:
df_bitcoin_processed.date = df_bitcoin_processed.date.astype(str)

### <span style='color:#ff5f27'> 💭 Tweets Data

In [ ]:
tweets_textblob = pd.read_csv("https://repo.hops.works/dev/davit/bitcoin/tweets_textblob.csv")
tweets_textblob.unix = tweets_textblob.unix.apply(tweets.fix_unix)

In [ ]:
tweets_vader = pd.read_csv("https://repo.hops.works/dev/davit/bitcoin/tweets_vader.csv")
tweets_vader.unix = tweets_vader.unix.apply(tweets.fix_unix)

In [ ]:
tweets_textblob.date = tweets_textblob.date.apply(lambda x: x[:10])
tweets_vader.date = tweets_vader.date.apply(lambda x: x[:10])

tweets_textblob.drop(tweets_textblob.columns[0],axis=1,inplace=True)
tweets_vader.drop(tweets_vader.columns[0],axis=1,inplace=True)

In [ ]:
tweets_textblob.tail(3)

In [ ]:
tweets_vader.tail(3)

---

## <span style="color:#ff5f27;"> 📡 Connecting to the Hopsworks Feature Store </span>

In [ ]:
import hopsworks

project = hopsworks.login()

fs = project.get_feature_store()

## <span style="color:#ff5f27;"> 🪄 Creating Feature Groups </span>

### <span style='color:#ff5f27'> 📈 Bitcoin Price Feature Group

In [ ]:
btc_price_fg = fs.get_or_create_feature_group(
    name='bitcoin_price',
    description='Bitcoin price aggregated for days',
    version=1,
    primary_key=['unix'],
    online_enabled=True,
    event_time='unix',
)

btc_price_fg.insert(df_bitcoin_processed)

### <span style='color:#ff5f27'> 💭 Tweets Feature Groups

In [ ]:
tweets_textblob_fg = fs.get_or_create_feature_group(
    name='bitcoin_tweets_textblob',
    version=1,
    primary_key=['unix'],
    online_enabled=True,
    event_time='unix',
)

tweets_textblob_fg.insert(tweets_textblob)

In [ ]:
tweets_vader_fg = fs.get_or_create_feature_group(
    name='bitcoin_tweets_vader',
    version=1,
    primary_key=['unix'],
    online_enabled=True,
    event_time='unix',
)

tweets_vader_fg.insert(
    tweets_vader,
    write_options={"wait_for_job": True},
)

## <span style="color:#ff5f27;">⏭️ **Next:** Part 02: Feature Pipeline</span>

In the next notebook you will be parsing new monthly data for the Feature Groups.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/logicalclocks/hopsworks-tutorials/blob/master/advanced_tutorials/bitcoin/2_bitcoin_feature_pipeline.ipynb)